In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random
import scipy.stats as stats

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy 

import scipy.interpolate as interpolate
import scipy.constants as constants

In [2]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

sys.path.append('/home/analysis_user/New_trap_code/SensitivityFramework/')
from signal_model_utilities_v3b import *

In [3]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

### Setup parameters

In [4]:
df = pd.read_pickle("/data/new_trap_processed/processed_files/20200320/Bead1/Shaking/Shaking378/Shaking3_aux.pkl") #here one could put the respective file name
height_cal = df["z_distance_emp"]
time = df['Time_Epoch']
time -= time[0]

In [5]:
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_X_m300k_250s_1hz.h5'
tf_x = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Y_m300k_250s_1hz.h5'
tf_y = BeadDataFile.BeadDataFile(fname)
fname = r'/home/data_transfer/data_from_windows/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
gfw.build_transfer_function([tf_x, tf_y, tf_z], base_freq=1, number_of_harmonics=90, scale_freq=71, plot=False)

No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data
No height data
No cantilever data
Error loading spinning data
No laser power data


In [6]:
bdf_discharge = load_dir(dirname='/data/new_trap/20200320/Bead1/Discharge/Discharge_after_Mass_20200402/After_spinning_again_20200403/',
                         start_file=230, max_file=40)
bdf_z = bdf_discharge[5:35]

530  files in folder
40  files loaded


In [7]:
bandwidth = 2
m1_zscale = gfw.build_z_response(bdf_z, drive_freq=71, charges=6, bandwidth=bandwidth, decimate=10)

***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  175.2406092149549
***************************************************
Z2-amplitude:  2.61e+02
reduced chi2:  392.6832937186789
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  233.0708308067062
***************************************************
Z2-amplitude:  2.79e+02
reduced chi2:  341.818416019317
***************************************************
Z2-amplitude:  2.81e+02
reduced chi2:  320.1151243984675
***************************************************
Z2-amplitude:  2.75e+02
reduced chi2:  235.2380260183528
***************************************************
Z2-amplitude:  2.70e+02
reduced chi2:  250.1490478570203
***************************************************
Z2-amplitude:  2.66e+02
reduced chi2:  270.7275591716337
***************************************************
Z2-amplitude:  2.67e+02
reduced chi2:  318.6342737410096
********************

In [8]:
print("check if scale is positive: ", gfw.scale_Z2)
# gfw.scale_Z2 *= -1

check if scale is positive:  1.8474324693e+17


In [9]:
def get_Yoffset(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return (437.3 - 25 * 9.5) - (aux.voltage_to_position(np.mean(bb.cant_pos[1])))
def get_Stroke(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return -np.std(aux.voltage_to_position(bb.cant_pos[1]))*np.sqrt(2)*2
def get_Dist(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)    
    return 392.2 - aux.voltage_to_position(np.mean(bb.cant_pos[0]))

In [136]:
z_sep = np.mean(height_cal)
y_offset = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Yoffset)(i) for i in tqdm(range(0, 10000, 100)))))
stroke = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Stroke)(i) for i in tqdm(range(0, 10000, 100)))))
dist = np.mean(np.array(Parallel(n_jobs=10)(delayed(get_Dist)(i) for i in tqdm(range(0, 10000, 100)))))
G_lambda = float(100)
print(z_sep, y_offset, stroke, dist, G_lambda)

100%|██████████| 100/100 [00:00<00:00, 673.44it/s]


-15.859100393 4.88214369679 -202.765748131 13.9887662292 100.0


In [137]:
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])

Loading Gravity Data... Done!
40 0.000101557361544
Loaded Yukawa Force


In [12]:
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = np.sqrt(2)*fft_norm(5000, 5000)

_,ax = plt.subplots()
x = shaking378_template
fft = np.abs(np.fft.rfft(x))*scale*1.85/1.55
freq = np.fft.rfftfreq(len(x), d=1./5000)
ax.semilogy(freq, fft, label='7.6 $\mu m$ sphere - Z force')
ax.set(xlim=(1,60), ylim=(None,None), xlabel='frequency [Hz]', ylabel='Force [N]', title=r'$\alpha=1x10^{8}$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
def func_filter(i):
    fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking378/Shaking3_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20200320/Bead1/Shaking/Shaking2_20200403/NoShaking_1_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20201211/Bead1/InitialTest/Data'+str(18+i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    res = bb.z2
    return (res)

In [13]:
test_phases = np.array(Parallel(n_jobs=46)(delayed(func_filter)(i) for i in tqdm(range(10000))))

100%|██████████| 10000/10000 [01:50<00:00, 90.36it/s]


In [14]:
def func_sideband(x, gravity_freqs_all):
    res = []
    for freq_ in gravity_freqs_all:
        b, a = signal.butter(3, [2.*(freq_+1.5-bandwidth/2.)/5000, 2.*(freq_+1.5+bandwidth/2.)/5000], btype = 'bandpass')
        responsefilt = signal.filtfilt(b, a, x)
        res.append(np.std(responsefilt))
    return np.array(res)

In [27]:
ll = likelihood_analyser.LikelihoodAnalyser()

def get_alpha(i):

    data_z2 = np.mean(test_phases[i:(i+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
              'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
              'limit_phi': [0, np.pi],
              'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
              'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
#     noises_std = np.array([1])
    noises_std = func_sideband(data_z2, gravity_freqs_all)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_mle_multiHarmoincs2(x=data_z2, template=shaking378_template, scales=gravity_scales, phases=gravity_phases, 
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    
#     PL_array = 2*10*ll.get_PL_multiHarmonics(A_array=A_array, **fit_kwargs)
#     PL_array -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PL_array))
        
    return mm_tmp.values[0]

In [55]:
A_array = np.arange(-5,5,0.1)


In [56]:
PLs_all_noise = np.array(Parallel(n_jobs=46)(delayed(get_alpha)(i) for i in tqdm(range(1000))))





  0%|          | 0/1000 [00:00<?, ?it/s]



  5%|▍         | 46/1000 [00:01<00:23, 39.82it/s]



  5%|▍         | 46/1000 [00:16<00:23, 39.82it/s]



  9%|▉         | 92/1000 [01:17<07:50,  1.93it/s]



 14%|█▍        | 138/1000 [02:34<12:23,  1.16it/s]



 18%|█▊        | 184/1000 [03:50<14:55,  1.10s/it]



 23%|██▎       | 230/1000 [05:05<16:11,  1.26s/it]



 28%|██▊       | 276/1000 [06:21<16:37,  1.38s/it]



 32%|███▏      | 322/1000 [06:43<12:29,  1.11s/it]

KeyboardInterrupt: 

In [28]:
PLtmp = []

In [33]:
A_array = np.arange(-1,1,0.01)
PLtmp.append(get_alpha(4))

MLEs:  <ValueView of Minuit at 7f9a800076e8>
  A: 0.0030165706848492846
  phi: 0.0
  sigma: 2.6676882882714636


In [68]:
PLs_summed_dig = []
PLscopy = copy.deepcopy(PLs_all_noise[0])
PLs_summed_dig = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_dig += pp_
PLs_summed_dig -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_dig))

In [69]:
_,ax = plt.subplots()
ax.plot(A_array*100, PLs_summed_dig, '*-')
ax.plot(A_array*100, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(-10,10), xlabel=r'$\alpha \, [10^{6}]$', ylabel = 'Profile-Likelihood', title='Digitization noise files ')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (-10.0, 10.0),
 Text(0.5, 0, '$\\alpha \\, [10^{6}]$'),
 Text(0, 0.5, 'Profile-Likelihood'),
 Text(0.5, 1.0, 'Digitization noise files ')]

In [161]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,13)[1:]
# gravity_freqs_all = np.array([36])
G_lambda = 10
test_shaking378 = force_vs_time(dist,z_sep,stroke,3,G_lambda,"z", 
                                yuk_or_grav="yuk",offset_y=y_offset, alpha=1e8, bead_size=7.6)
shaking378_template = np.array(test_shaking378[1])


fft_angles = np.angle(np.fft.rfft(shaking378_template))
fft_fft = np.abs(np.fft.rfft(shaking378_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./5000)
fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    
    gravity_phases.append(phase_all)

gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

Loading Gravity Data... Done!
81 9.74070978211e-06
Loaded Yukawa Force
No height data
No cantilever data
Error loading spinning data
No laser power data


In [41]:
mm_gravity = get_alpha(0)

MLEs:  <ValueView of Minuit at a6db948>
  A: 1.554781157465186
  phi: 0.0
  sigma: 812.7864802703888


### EDM settings

In [16]:
base_path = r'/home/nadavp/tmp/'
fname = 'new_trap_attractor_shield_potential_efield.txt'


positions_to_sample = np.array([
                                [13.9e-6, 0.0e-6, -15.23e-6], \
                               ])

dipole_units = constants.elementary_charge * 1.0e-6
dipole_moment = np.array([0.0, 0.0, 100.0]) * dipole_units


In [17]:
#############################################################
######  LOAD DATA FROM COMSOL OUTPUT TO NUMPY ARRAYS  #######
#############################################################

path = os.path.join(base_path, fname)

with open(path, 'r') as f:
    lines  =  f.readlines()

for lineind, line in enumerate(lines):
    if 'Grid' in line:
        grid_ind = lineind + 1
    elif 'V (V)' in line:
        voltage_ind = lineind + 1
    elif 'es.Ex (V/m)' in line:
        ex_ind = lineind + 1
    elif 'es.Ey (V/m)' in line:
        ey_ind = lineind + 1
    elif 'es.Ez (V/m)' in line:
        ez_ind = lineind + 1

xvec_str = lines[grid_ind].strip('\n')
yvec_str = lines[grid_ind+1].strip('\n')
zvec_str = lines[grid_ind+2].strip('\n')

xvec = np.array( [float(val) for val in xvec_str.split()] )
yvec = np.array( [float(val) for val in yvec_str.split()] )
zvec = np.array( [float(val) for val in zvec_str.split()] )

lenx = len(xvec)
leny = len(yvec)
lenz = len(zvec)

voltage = np.zeros((lenx, leny, lenz))
efieldx = np.zeros((lenx, leny, lenz))
efieldy = np.zeros((lenx, leny, lenz))
efieldz = np.zeros((lenx, leny, lenz))

for j in range(leny):
    for k in range(lenz):
        voltage_line = lines[voltage_ind + j + k*leny]
        voltage[:,j,k] = np.array( [float(val) for val in voltage_line.strip('\n').split()] ) 

        efieldx_line = lines[ex_ind + j + k*leny]
        efieldx[:,j,k] = np.array( [float(val) for val in efieldx_line.strip('\n').split()] )

        efieldy_line = lines[ey_ind + j + k*leny]
        efieldy[:,j,k] = np.array( [float(val) for val in efieldy_line.strip('\n').split()] )

        efieldz_line = lines[ez_ind + j + k*leny]
        efieldz[:,j,k] = np.array( [float(val) for val in efieldz_line.strip('\n').split()] )





#############################################################
##############  BUILD 3D GRID INTERPOLATORS  ################
#############################################################

voltage_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), voltage)

efieldx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldx)
efieldy_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldy)
efieldz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldz)

In [22]:
dipole_moment = np.array([100.0, 0.0, 0.0]) * dipole_units
x_att = [100*np.sin(3*t/500*2*np.pi) for t in range(500)]
positions_to_sample4 = np.array([[13.9*1e-6, (offset_-4.9)*1e-6, -15.23e-6] for offset_ in x_att])

In [23]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x4 = []
force_z4 = []
for point in positions_to_sample4:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x4.append(force_on_dipole[0])
    force_z4.append(force_on_dipole[2])


    MS dipole [e um] :  [ 100.    0.    0.]

    MS Postion [um]  :  [ 13.9   -4.9  -15.23]
        Efield [V/m] :  [-3629.04376377   -32.38996578   745.93786973]
Force on dipole [N]  :  [  9.17323934e-16   1.74836754e-16  -2.84188688e-15]

    MS Postion [um]  :  [ 13.9         -1.13098173 -15.23      ]
        Efield [V/m] :  [-3613.33808298   -18.74908864   743.87823202]
Force on dipole [N]  :  [  1.26241019e-15  -2.62847949e-16  -2.71042239e-15]

    MS Postion [um]  :  [ 13.9          2.63268055 -15.23      ]
        Efield [V/m] :  [-3775.95027138    79.85385886   689.03226592]
Force on dipole [N]  :  [  1.15575857e-15  -2.91496982e-16  -2.92504819e-15]

    MS Postion [um]  :  [ 13.9          6.38563849 -15.23      ]
        Efield [V/m] :  [-3560.1701477     28.00474276   811.18059762]
Force on dipole [N]  :  [ -3.15657994e-16  -5.89111969e-16  -3.48857521e-15]

    MS Postion [um]  :  [ 13.9         10.12255891 -15.23      ]
        Efield [V/m] :  [-3586.5531662     -5.99602

    MS Postion [um]  :  [ 13.9        94.5951017 -15.23     ]
        Efield [V/m] :  [-3579.1008388      9.03673029   801.38592255]
Force on dipole [N]  :  [  5.78343501e-16  -1.52369732e-16  -2.82548757e-15]

    MS Postion [um]  :  [ 13.9         94.90267284 -15.23      ]
        Efield [V/m] :  [ -3.58475748e+03   5.63826311e-01   7.89578818e+02]
Force on dipole [N]  :  [  7.38840560e-16  -1.15912741e-16  -2.80968662e-15]

    MS Postion [um]  :  [ 13.9         95.06841893 -15.23      ]
        Efield [V/m] :  [ -3.58610225e+03  -2.54027245e+00   7.85677760e+02]
Force on dipole [N]  :  [  8.20348145e-16  -9.06663918e-17  -2.78803272e-15]

    MS Postion [um]  :  [ 13.9         95.09210442 -15.23      ]
        Efield [V/m] :  [ -3.58594813e+03  -2.68669107e+00   7.85620694e+02]
Force on dipole [N]  :  [  8.30982979e-16  -8.59202267e-17  -2.78226743e-15]

    MS Postion [um]  :  [ 13.9         94.97369566 -15.23      ]
        Efield [V/m] :  [ -3.58606369e+03  -1.39269505e+00   7.8

    MS Postion [um]  :  [ 13.9         11.36371652 -15.23      ]
        Efield [V/m] :  [ -3.60070624e+03   2.46998204e+00   8.33064308e+02]
Force on dipole [N]  :  [  7.59139264e-16  -2.95061655e-16  -3.47267490e-15]

    MS Postion [um]  :  [ 13.9          7.63332336 -15.23      ]
        Efield [V/m] :  [-3595.17229866   -46.50943258   758.55351445]
Force on dipole [N]  :  [ -1.93366154e-17  -1.88750335e-16  -3.12193554e-15]

    MS Postion [um]  :  [ 13.9          3.88511966 -15.23      ]
        Efield [V/m] :  [-3789.63764482  -195.09211884   930.66377068]
Force on dipole [N]  :  [  1.51122577e-15   1.59022074e-15  -4.37756029e-15]

    MS Postion [um]  :  [ 13.9          0.12443182 -15.23      ]
        Efield [V/m] :  [-3631.35270529     5.28589288   746.28098593]
Force on dipole [N]  :  [  1.27558016e-15  -3.35199899e-16  -2.70776646e-15]

    MS Postion [um]  :  [ 13.9         -3.64339601 -15.23      ]
        Efield [V/m] :  [-3616.73768798   -30.83029278   741.90376849]
Fo

    MS Postion [um]  :  [  13.9        -102.35268728  -15.23      ]
        Efield [V/m] :  [-3417.39986295   -61.42062823   635.70489757]
Force on dipole [N]  :  [  1.33812128e-15  -2.66414204e-16  -3.30463597e-15]

    MS Postion [um]  :  [  13.9        -103.12872507  -15.23      ]
        Efield [V/m] :  [-3387.96236136   -63.61128224   668.23887155]
Force on dipole [N]  :  [  7.88033053e-16  -3.19864406e-16  -3.28762337e-15]

    MS Postion [um]  :  [  13.9        -103.76517447  -15.23      ]
        Efield [V/m] :  [-3369.0966476    -76.98210176   659.16467693]
Force on dipole [N]  :  [  7.22222624e-16  -2.65286137e-16  -3.30158718e-15]

    MS Postion [um]  :  [  13.9        -104.26113105  -15.23      ]
        Efield [V/m] :  [-3365.80549671   -80.70798187   656.19162997]
Force on dipole [N]  :  [  5.95228054e-16  -1.90668162e-16  -3.24866064e-15]

    MS Postion [um]  :  [  13.9        -104.61589003  -15.23      ]
        Efield [V/m] :  [-3370.79079202   -79.06761475   656.701

    MS Postion [um]  :  [ 13.9        -33.40192625 -15.23      ]
        Efield [V/m] :  [-3560.34038986   -65.60288661   746.23608462]
Force on dipole [N]  :  [  3.09252695e-16   1.88505736e-16  -3.28425722e-15]

    MS Postion [um]  :  [ 13.9        -29.76898872 -15.23      ]
        Efield [V/m] :  [-3493.58389895   -15.41187987   795.01175276]
Force on dipole [N]  :  [  3.42349926e-16   3.01692928e-16  -4.12690675e-15]

    MS Postion [um]  :  [ 13.9        -26.10071099 -15.23      ]
        Efield [V/m] :  [-3481.72094555   -34.27123803   837.93960662]
Force on dipole [N]  :  [  4.70208087e-16   6.69482823e-17  -4.02885779e-15]

    MS Postion [um]  :  [ 13.9       -22.4023059 -15.23     ]
        Efield [V/m] :  [-3603.42652995   -40.60965082   766.23705502]
Force on dipole [N]  :  [  5.35464626e-16  -9.36571669e-17  -3.93750617e-15]

    MS Postion [um]  :  [ 13.9        -18.67902907 -15.23      ]
        Efield [V/m] :  [-3591.09439558   -16.41534073   750.31233761]
Force on di

...

    MS Postion [um]  :  [ 13.9        -56.34395338 -15.23      ]
        Efield [V/m] :  [-3640.49419989   -49.80195663   677.85759035]
Force on dipole [N]  :  [  9.47325825e-16  -7.83395098e-18  -3.11214273e-15]

    MS Postion [um]  :  [ 13.9        -53.07536741 -15.23      ]
        Efield [V/m] :  [-3588.62998889   -51.05343456   712.43788393]
Force on dipole [N]  :  [  7.46499273e-16  -2.93496679e-17  -3.01937134e-15]

    MS Postion [um]  :  [ 13.9        -49.73832161 -15.23      ]
        Efield [V/m] :  [-3594.13527096   -64.56767859   715.59361748]
Force on dipole [N]  :  [  1.04535927e-15  -1.36793008e-16  -2.78396198e-15]

    MS Postion [um]  :  [ 13.9       -46.3375581 -15.23     ]
        Efield [V/m] :  [-3553.97853439   -41.13162169   706.99913479]
Force on dipole [N]  :  [  1.39278574e-15   1.75456862e-16  -2.97107701e-15]

    MS Postion [um]  :  [ 13.9        -42.87790955 -15.23      ]
        Efield [V/m] :  [-3529.40508599     4.94511528   670.86226679]
Force on di

    MS Postion [um]  :  [ 13.9         77.45325976 -15.23      ]
        Efield [V/m] :  [-3543.54560902    14.20298581   772.40606065]
Force on dipole [N]  :  [  8.49461634e-16  -1.15835838e-17  -2.67971064e-15]

    MS Postion [um]  :  [ 13.9         79.53279255 -15.23      ]
        Efield [V/m] :  [-3556.00725354     6.16257933   791.65302661]
Force on dipole [N]  :  [  1.01302180e-15  -2.20514897e-16  -2.86618475e-15]

    MS Postion [um]  :  [ 13.9         81.49234172 -15.23      ]
        Efield [V/m] :  [-3560.06819512    22.53082555   821.87032546]
Force on dipole [N]  :  [  6.58861910e-16   9.46164661e-17  -2.92231780e-15]

    MS Postion [um]  :  [ 13.9         83.32912264 -15.23      ]
        Efield [V/m] :  [-3555.38576688    16.24829797   840.53673344]
Force on dipole [N]  :  [  7.31820370e-16  -1.34059801e-16  -2.99794317e-15]

    MS Postion [um]  :  [ 13.9         85.04052516 -15.23      ]
        Efield [V/m] :  [-3572.27884286    18.76000451   833.46509963]
Force on

    MS Postion [um]  :  [ 13.9         56.88596131 -15.23      ]
        Efield [V/m] :  [ -3.64199249e+03  -3.23561943e-01   8.37503280e+02]
Force on dipole [N]  :  [  1.24604653e-15   1.25484585e-16  -3.23247017e-15]

    MS Postion [um]  :  [ 13.9         53.87852523 -15.23      ]
        Efield [V/m] :  [-3678.30779216    26.07094068   854.47946414]
Force on dipole [N]  :  [  1.57127907e-15   3.14605766e-18  -2.99433152e-15]

    MS Postion [um]  :  [ 13.9         50.78756165 -15.23      ]
        Efield [V/m] :  [-3659.29045266    21.82336055   853.23524259]
Force on dipole [N]  :  [  1.55846345e-15  -1.12726016e-16  -2.88813349e-15]

    MS Postion [um]  :  [ 13.9       47.617463 -15.23    ]
        Efield [V/m] :  [-3637.20564451     6.51425718   860.13378983]
Force on dipole [N]  :  [  1.67035584e-15   1.51341069e-18  -3.47525126e-15]

    MS Postion [um]  :  [ 13.9         44.37273415 -15.23      ]
        Efield [V/m] :  [-3645.14935177   -47.17909536   917.36247474]
Force on

    MS Postion [um]  :  [ 13.9        -79.49411454 -15.23      ]
        Efield [V/m] :  [-3453.82752197   -15.01416303   657.84847385]
Force on dipole [N]  :  [  1.00316380e-15  -3.65246350e-16  -2.66083510e-15]

    MS Postion [um]  :  [ 13.9        -81.95132428 -15.23      ]
        Efield [V/m] :  [-3401.9782076    -34.31383216   632.78986525]
Force on dipole [N]  :  [  1.80938128e-15  -1.49773849e-16  -3.21720660e-15]

    MS Postion [um]  :  [ 13.9        -84.29903986 -15.23      ]
        Efield [V/m] :  [-3399.04788037   -58.06180758   638.46776036]
Force on dipole [N]  :  [  1.95803115e-15   1.32256864e-16  -3.14723823e-15]

    MS Postion [um]  :  [ 13.9        -86.53392507 -15.23      ]
        Efield [V/m] :  [-3453.73228199   -56.46048794   670.94689303]
Force on dipole [N]  :  [  1.83739884e-15   1.56456443e-16  -3.08748380e-15]

    MS Postion [um]  :  [ 13.9      -88.652804 -15.23    ]
        Efield [V/m] :  [-3439.72389444   -18.00542233   704.49813684]
Force on dipol

    MS Postion [um]  :  [ 13.9        -76.05356772 -15.23      ]
        Efield [V/m] :  [-3516.23200559  -118.56255489   673.5812904 ]
Force on dipole [N]  :  [  1.45915012e-15   2.80692274e-16  -2.53320610e-15]

    MS Postion [um]  :  [ 13.9        -73.35471059 -15.23      ]
        Efield [V/m] :  [-3577.93944415   -17.52074085   727.67184763]
Force on dipole [N]  :  [  1.99891452e-15  -2.50816937e-16  -2.89927827e-15]

    MS Postion [um]  :  [ 13.9        -70.55857558 -15.23      ]
        Efield [V/m] :  [-3591.02008938   -24.68023752   690.58626274]
Force on dipole [N]  :  [  1.81328320e-15  -2.78804771e-16  -2.98118902e-15]

    MS Postion [um]  :  [ 13.9        -67.66913613 -15.23      ]
        Efield [V/m] :  [-3641.26085189    31.88477968   668.73012357]
Force on dipole [N]  :  [  1.28533419e-15   4.46106245e-16  -3.65245011e-15]

    MS Postion [um]  :  [ 13.9        -64.69049831 -15.23      ]
        Efield [V/m] :  [-3524.04375984    23.98628862   672.56609738]
Force on

In [24]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(500), force_z4)
# ax.scatter(range(500), force_z4[::10])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(3,13)

shaking378_template = np.array(test_shaking378[1])
shaking378_template = force_z3

fft_angles = np.angle(np.fft.rfft(shaking378_template))
fft_fft = np.abs(np.fft.rfft(shaking378_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(shaking378_template), d=1./500)
# fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
# tf_z = BeadDataFile.BeadDataFile(fname)
fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

gravity_phases = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    gravity_phases.append(phase_all)

gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

In [28]:
mm_gravity2_2 = get_alpha(0)

MLEs:  <ValueView of Minuit at 7f1998d67508>
  A: -11.896177522474318
  phi: 0.0
  sigma: 930.7935119307609


In [114]:
mm_gravity.fval, mm_gravity2.fval, mm_gravity2_2.fval

(129.02309291499253, 109.20476540830566, 109.22296265489145)

### Test new fitter

In [37]:
ll = likelihood_analyser.LikelihoodAnalyser()
def get_edm(i):

    data_z2 = np.mean(test_phases[i:(i+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'error_A': 0.1, 'limit_A': [-10, 10],
                  'A2': 0.0, 'error_A2': 0.1, 'limit_A2': [-10, 10],
                  'A3': 0.0, 'error_A3': 0.1, 'limit_A3': [-10, 10],
                  'A4': 0.0, 'error_A4': 0.1, 'limit_A4': [-100, 100],
                  'sigma':840, 'error_sigma': 10, 'limit_sigma': [0, 1000000],
               'errordef': 1, 'print_level': 0, 
                  'fix_A': False, 'fix_A2': False, 'fix_A3': False, 'fix_A4': False, 'fix_sigma': False}
    
#     fit_kwargs = {'A': 0, 'phi': 0, 'sigma':13.85,
#               'error_A': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma': 10,
#               'limit_phi': [0, np.pi],
#               'limit_A': [-100000, 100000], 'limit_sigma': [0, 1000000],
#               'print_level': 0, 'fix_A': False, 'fix_phi': True, 'fix_sigma': False}
    
    noises_std = func_sideband(data_z2, gravity_freqs_all)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_edm_multiHarmoincs2(x=data_z2, amps1=edm_amp4, phases1=edm_phases4, 
                                         amps2=edm_amp5, phases2=edm_phases5, 
                                         amps3=edm_amp6, phases3=edm_phases6, 
                                         amps4=gravity_amp, phases4=gravity_phases, scales=gravity_scales,
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
        
    return mm_tmp.values[0], mm_tmp.values[1], mm_tmp.values[2], mm_tmp.values[3], mm_tmp.fval

In [168]:
len(gravity_amp), len(edm_amp4), len(edm_amp5), len(edm_amp6)

(6, 6, 6, 6)

In [356]:
index = np.random.choice(11, 11, replace=False)
edm_amp4 = [edm_amp[i] for i in index]
index = np.random.choice(11, 11, replace=False)
edm_phases4 = [np.random.uniform()*2*np.pi for i in index]

In [179]:
## likelihood calculator stuff
gravity_freqs_all = 3*np.arange(1,13)[1:]
# gravity_freqs_all = np.array([12, 18, 21, 33, 36, 39])

edm_template = np.array(test_shaking378[1])
# edm_template = force_z4

fft_angles = np.angle(np.fft.rfft(edm_template))
fft_fft = np.abs(np.fft.rfft(edm_template)) * 2 / np.sqrt(5000 * 5000)
freq1 = np.fft.rfftfreq(len(edm_template), d=1./5000)
# fname = r'/data/new_trap/20200320/Bead1/TransFunc/Repeat_TransFunc_20200104/TransFunc_Z_m300k_250s_1hz.h5'
# tf_z = BeadDataFile.BeadDataFile(fname)
# fft_angles2 = np.angle(np.fft.rfft(tf_z.z2))
# freq2 = np.fft.rfftfreq(len(tf_z.z2), d=1./5000)
# fft_angles3 = np.angle(np.fft.rfft(tf_z.electrode_data[0]))
# freq3 = np.fft.rfftfreq(len(tf_z.electrode_data[0]), d=1./5000)

# edm_phases4 = []
# edm_amp4 = []

gravity_phases = []
gravity_amp = []
for drive_freq in gravity_freqs_all:
    phase_th = fft_angles[freq1==drive_freq]  ## theory 
    phase_th2 = fft_angles2[freq2==drive_freq] ## response phase
    phase_th3 = fft_angles3[freq3==drive_freq] ## drive phase
    phase_all = (phase_th+(phase_th2-phase_th3)+np.pi/2)%(2*np.pi)
    gravity_phases.append(phase_all)
    gravity_amp.append(fft_fft[freq1==drive_freq])
gravity_scales = np.array([gfw.scale_Z2])* np.interp(gravity_freqs_all, gfw.tf_freq, gfw.tf_ffts[2])
# gravity_phases = np.vstack(gravity_phases).T

In [139]:
i=100
get_edm(i)

MLEs:  <ValueView of Minuit at 423ac0f8>
  A: -0.009461160021640325
  A2: -0.03695863756674811
  A3: -0.0035185586649273404
  A4: -0.25688980178018994
  sigma: 967.1782015661101


(-0.009461160021640325,
 -0.03695863756674811,
 -0.0035185586649273404,
 -0.25688980178018994,
 162.25230842568374)

In [38]:
mm_all_edm_signal = np.array(Parallel(n_jobs=46)(delayed(get_edm)(i) for i in tqdm(range(10000))))

100%|██████████| 10000/10000 [06:53<00:00, 24.21it/s]


In [117]:
edm1_all = [mm_.values[0] for mm_ in mm_all_only1]
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(len(edm1_all)), edm1_all)
ax.scatter(range(len(mm_all_only2)), mm_all_only2)
ax.scatter(range(len(mm_all_only3)), mm_all_only3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
# edm1_all2 = [mm_.values[0] for mm_ in mm_all_only1 if (mm_.values[0]<2)&(mm_.values[0]>-2)]
# edm2_all2 = [mm_ for mm_ in mm_all_only2 if (mm_<2)&(mm_>-2)]
# edm3_all2 = [mm_ for mm_ in mm_all_only3 if (mm_<2)&(mm_>-2)]
fig,ax = plt.subplots(1,2,figsize=(9.2,4))
_ = ax[0].hist(mm_all_edm_signal[:,0], bins=50, range=(-2,2), label=r'$p_x$ scale')
_ = ax[0].hist(mm_all_edm_signal[:,1], bins=20, alpha=0.7, range=(-2,2), label=r'$p_y$ scale')
_ = ax[0].hist(mm_all_edm_signal[:,2], bins=50, alpha=0.4, range=(-2,2), label=r'$p_z$ scale')
ax[0].set(xlabel='simulation scale factor')
ax[0].legend()
_ = ax[1].hist(np.sqrt(mm_all_edm_signal[:,0]**2+mm_all_edm_signal[:,1]**2+mm_all_edm_signal[:,2]**2), alpha=0.4, range=(0,5), bins=40, 
               label=r'$\sqrt{p_x^2+p_y^2+p_z^2}$')
ax[1].set(xlabel=r'simulation scale factor')
ax[1].legend()

# print(np.mean(mm_all_edm[:,0]), np.std(mm_all_edm[:,0]))
# print(np.mean(mm_all_edm[:,1]), np.std(mm_all_edm[:,1]))
# print(np.mean(mm_all_edm[:,2]), np.std(mm_all_edm[:,2]))
# print(np.mean(np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2)))
# print(np.std(np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2)))
# fig.savefig("edm_only_distribution.png", format='png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [98]:
_,ax = plt.subplots(figsize=(9.2,4))
titles = [r'$p_x$ scale',r'$p_y$ scale',r'$p_z$ scale']
[ax.scatter(range(10000), mm_all_edm[:,i],label=titles[i]) for i in range(3)]
ax.legend()
ax.set(ylabel='simulation scale', xlabel='dataset')
# ax.scatter(range(10000), np.sqrt(mm_all_edm[:,0]**2+mm_all_edm[:,1]**2+mm_all_edm[:,2]**2))
# ax.scatter(range(1000), mm_all_edm3[:,3])
# ax.scatter(range(10000), mm_all_edm[:,4])
# ax.scatter(range(10000), mm_all_edm5[:,4])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.5, 'simulation scale'), Text(0.5, 0, 'dataset')]

In [184]:
mm_all_edm4.shape

(10000, 5)

In [359]:
alpha_edm = [alpha_ for alpha_,chi2 in zip(mm_all_edm[:,3],mm_all_edm[:,4]) if chi2<180]
alpha_edm5 = [alpha_ for alpha_,chi2 in zip(mm_all_edm5[:,3],mm_all_edm5[:,4]) if chi2<170]
alpha_edm11 = [alpha_ for alpha_,chi2 in zip(mm_all_edm11[:,3],mm_all_edm11[:,4]) if chi2<170]

In [360]:
len(alpha_edm11)

9705

In [527]:
_,ax = plt.subplots(figsize=(9.2,4))
ax.scatter(range(len(alpha_edm)), alpha_edm, label='with edm')
ax.scatter(range(len(alpha_edm5)), alpha_edm5, alpha=0.7, label='w/o edm')
ax.scatter(range(len(alpha_edm11)), alpha_edm11, alpha=0.4, label='random edm')
ax.legend()
ax.set(ylabel='simulation scale', xlabel='dataset')
print('with edm (mean, ste): ',np.mean(mm_all_edm[:,3]), np.std(mm_all_edm[:,3])/np.sqrt(10000))
print('w/o edm (mean, ste): ',np.mean(mm_all_edm5[:,3]), np.std(mm_all_edm5[:,3])/np.sqrt(10000))
print('random edm (mean, ste): ',np.mean(mm_all_edm11[:,3]), np.std(mm_all_edm11[:,3])/np.sqrt(10000))
# print('with edm (mean, ste): ', np.mean(alpha_edm), np.std(alpha_edm)/np.sqrt(10000))
# print(np.mean(alpha_edm5), np.std(alpha_edm5)/np.sqrt(10000))
# print(np.mean(alpha_edm11), np.std(alpha_edm11)/np.sqrt(10000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

with edm (mean, ste):  0.314726936746 0.147261649134
w/o edm (mean, ste):  0.830508937732 0.112488776049
random edm (mean, ste):  0.864447292648 0.148614968952


In [56]:
import matplotlib as mpl 
mpl.rcParams['hatch.color'] = 'blue'  # previous svg hatch linewidth

In [134]:
np.mean(mm_all_signal[:,3]), np.mean(mm_all_edm_signal[:,3]), np.std(mm_all_edm_signal[:,3])/np.sqrt(10000)

(0.78377068463144739, 0.44581993432240297, 0.095350057158780074)

In [182]:
fig,ax = plt.subplots()
# ax.hist(mm_all_edm11[:,3], range=(-25,25), bins=30, alpha=0.7, label='random edm')

mpl.rcParams['hatch.color'] = 'red'  # previous svg hatch linewidth
ax.hist(mm_all_signal[:,3], range=(-25,25), color='red', bins=30, hatch=r'\\', alpha=0.2, label='no edm', density=True)
mpl.rcParams['hatch.color'] = 'blue'  # previous svg hatch linewidth
ax.hist(mm_all_edm_signal[:,3], range=(-25,25), bins=30, hatch='//', alpha=0.2, label='with edm', density=True)
ax.legend()
ax.set_xlabel(r'$\hat \alpha \, [10^{8}]$', size=15)

from scipy.stats import norm
# Fit a normal distribution to the data:
mu, std = norm.fit(mm_all_edm_signal[:,3])
x = np.linspace(-25, 25, 100)
p = norm.pdf(x, mu, std)
ax.plot(x, p, '--b', linewidth=2, label='with edm')

mu2, std2 = norm.fit(mm_all_signal[:,3])
p = norm.pdf(x, mu2, std2)
ax.plot(x, p, '--r', linewidth=2, label='no edm')
# ax.legend()
fig.savefig("alpha_distribution_v3.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### PL

In [495]:
index = np.random.choice(10000, 10000, replace=False)

In [180]:
ll = likelihood_analyser.LikelihoodAnalyser()

def calc_PL(i):

#     ii = index[i]
    ii = i
    data_z2 = np.mean(test_phases[ii:(ii+1)], axis=0)
    
    fit_kwargs = {'A': 0, 'error_A': 0.1, 'limit_A': [-10, 10],
                  'A2': 0.0, 'error_A2': 0.1, 'limit_A2': [-10, 10],
                  'A3': 0.0, 'error_A3': 0.1, 'limit_A3': [-10, 10],
                  'A4': 0.0, 'error_A4': 0.1, 'limit_A4': [-100, 100],
                  'sigma':13.85, 'error_sigma': 10, 'limit_sigma': [0, 1000000],
               'errordef': 1, 'print_level': 0, 
                  'fix_A': False, 'fix_A2': False, 'fix_A3': False, 'fix_A4': False}
    
    noises_std = func_sideband(data_z2, gravity_freqs_all)
    noises_std = noises_std/np.min(noises_std)

    mm_tmp = ll.find_edm_multiHarmoincs2(x=data_z2, amps1=edm_amp4, phases1=edm_phases4, 
                                         amps2=edm_amp5, phases2=edm_phases5, 
                                         amps3=edm_amp6, phases3=edm_phases6, 
                                         amps4=gravity_amp, phases4=gravity_phases, scales=gravity_scales,
                                          signal_freqs=gravity_freqs_all, bandwidth=bandwidth, noises=noises_std,
                                          decimate=10, **fit_kwargs)
    print('MLEs: ', mm_tmp.values)
    

    fit_kwargs['A'] = mm_tmp.values[0]
    fit_kwargs['A2'] = mm_tmp.values[1]
    fit_kwargs['A3'] = mm_tmp.values[2]
    fit_kwargs['sigma'] = mm_tmp.values[4]
    fit_kwargs['fix_sigma'] = True
    
    PL_array = A_array*0
    PL_array = 2*10*ll.get_PL_edm_multiHarmonics(A_array=A_array, **fit_kwargs)
        
    return PL_array

In [181]:
A_array = np.arange(-1,1,0.01)
PLs = []
for i in range(1):
    print('****************************')
    print(i)
    print('****************************')
    PLs.append(np.array(Parallel(n_jobs=40)(delayed(calc_PL)(i) for i in tqdm(range(1000)))))


  0%|          | 0/1000 [00:00<?, ?it/s]

****************************
0
****************************



100%|██████████| 1000/1000 [56:05<00:00,  3.60s/it]

In [ ]:
len(PLs[0][0])

In [173]:
PLs_summed_harmonics = []
PLscopy = copy.deepcopy(PLs[0])
PLs_summed_harmonics = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_harmonics += pp_
PLs_summed_harmonics -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_harmonics))

# PLs_summed_harmonics2 = []
# PLscopy = copy.deepcopy(PLs2)
# PLs_summed_harmonics2 = PLscopy[0]
# for pp_ in PLscopy[1:]:
#     PLs_summed_harmonics2 += pp_
# PLs_summed_harmonics2 -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_harmonics2))

In [175]:
_,ax = plt.subplots()
# i=10
# [ax.plot(A_array, PLs[i], '*-') for i in range(100)]
ax.plot(A_array*10, PLs_summed_harmonics, '*-', label='with edm')
# ax.plot(A_array, PLs_summed_harmonics2, '*-', label='no edm')
# ax.legend()
ax.plot(A_array*10, A_array*0 + 2.71, 'r--')
# ax.set(ylim=(0,10), xlim=(-10,10), xlabel=r'$\alpha \, [10^{8}]$', ylabel = 'Profile-Likelihood')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [588]:
type(PLs)

numpy.ndarray

In [621]:
PLlist = []
for i in range(10):
    PLlist += list(PLs_all_noEDM[i])

PLs_summed_noedm = []
PLscopy = copy.deepcopy(np.array(PLlist))
PLs_summed_noedm = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_noedm += pp_
PLs_summed_noedm -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_noedm))

PLlist2 = []
for i in range(10):
    PLlist2 += list(PLs_all_EDM[i])

PLs_summed_edm = []
PLscopy = copy.deepcopy(np.array(PLlist2))
PLs_summed_edm = PLscopy[0]
for pp_ in PLscopy[1:]:
    PLs_summed_edm += pp_
PLs_summed_edm -= np.min(np.interp(np.arange(np.min(A_array),np.max(A_array),0.0001), A_array, PLs_summed_edm))


In [628]:
_,ax = plt.subplots()
ax.plot(A_array*10, PLs_summed_edm, '*-', label='with edm')
ax.plot(A_array*10, PLs_summed_noedm, '*-', label='no edm')
ax.legend()
ax.plot(A_array*10, A_array*0 + 2.71, 'r--')
ax.set(ylim=(0,10), xlim=(-8,8), xlabel=r'$\alpha \, [10^{7}]$', ylabel = 'Profile-Likelihood', title='all files')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[(0.0, 10.0),
 (-8.0, 8.0),
 Text(0.5, 0, '$\\alpha \\, [10^{7}]$'),
 Text(0, 0.5, 'Profile-Likelihood'),
 Text(0.5, 1.0, 'all files')]

In [167]:
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = fft_norm(5000, 5000)

fig,ax = plt.subplots()
fft_x = np.abs(np.fft.rfft(force_x4))*scale
fft_z = np.abs(np.fft.rfft(force_z4))*scale

freqs_gravity = 3*np.arange(14)
z_to_x = np.array([fft_z[freq1==freq_]/fft_x[freq1==freq_] for freq_ in freqs_gravity])

freq1 = np.fft.rfftfreq(len(force_x4), d=1./5000)
ax.semilogy(freq1, fft_x, label='x-backgrounds')
ax.semilogy(freq1, fft_z, label='z-backgrounds')
ax.set(xlim=(1,70), ylim=(1e-19,1e-14), xlabel='frequency [Hz]', ylabel=r'ASD [N/$\sqrt{Hz.}$]')
ax.legend()
fig.savefig('x_dipole_backgrounds_template.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: __delete__